In [3]:
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter


class CNNnetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1d = nn.Conv1d(1, 64, kernel_size=2)
        self.relu = nn.ReLU(inplace=True)
        self.Linear1 = nn.Linear(64 * 8, 50)
        self.Linear2 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(-1)
        x = self.Linear1(x)
        x = self.relu(x)
        x = self.Linear2(x)
        return x


torch.Size([1])


In [ ]:
import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler


Datasets = pd.read_csv('data/Google_Stock_Price_Train.csv')
data = Datasets['Open'].values.astype(float)
train_set = data[:-int(len(data) * 0.2)]
test_set = data[-int(len(data) * 0.2):]
# print(len(train_set))   # 202
# print(len(test_set))    # 50

scaler = MinMaxScaler(feature_range=(-1, 1))
train_norm = scaler.fit_transform(train_set.reshape(-1, 1))
train_norm = torch.FloatTensor(train_norm).view(-1)
train_data = []
seq_size = 9
for i in range(len(train_norm) - seq_size):
    window = train_norm[i:i + seq_size]
    label = train_norm[i + seq_size]
    train_data.append((window, label))

mod = CNNnetwork()

loss_fn = nn.MSELoss()

learning_rate = 1e-2
optimizer = torch.optim.SGD(mod.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(mod.parameters(), lr=learning_rate)
total_train_step = 0
epoch = 500
loss_list = []
mod.train()
for i in range(epoch):
    for seq, y_true in train_data:
        optimizer.zero_grad()
        out = mod(seq.reshape(1, 1, -1))
        loss = loss_fn(out, y_true)
        loss.backward()
        optimizer.step()
        total_train_step = total_train_step + 1
        if total_train_step % 1000 == 0:
            print('训练次数:{}，loss:{}'.format(total_train_step, loss.item()))
            loss_list.append(loss.item())

preds = train_norm[-seq_size:].tolist()
mod.eval()
for i in range(52):
    seq = torch.FloatTensor(preds[-seq_size:])
    with torch.no_grad():
        preds.append(mod(seq.reshape(1, 1, -1)).item())
print(preds)
true_value = scaler.inverse_transform(np.array(preds[seq_size:]).reshape(-1, 1))
print(true_value.tolist())
print(test_set)
plt.figure(1)
plt.plot(test_set, label='true value')
plt.plot(true_value, label='predict value')
plt.legend(loc="upper left")
plt.figure(2)
plt.plot(loss_list, label='loss')
plt.show()


CNN-LSTM

In [9]:
import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [16]:
para_dict = {
        "input_size":1,
        "output_size":1,
        "batch_size":5,
        "hidden_size":64,
        "num_layers":3,
        "seq_len":7,
        "modelpara_path":'md.pth',
        "loss_function":'mse',
        "optimizer":'Adam',
        "lr":0.0001,
        "epoch":20,
        "in_channels":7,
        "out_channels":3,
        "kernel_size":3
    }

In [17]:
class CNN_LSTM(nn.Module):
    def __init__(self, args):
        super(CNN_LSTM, self).__init__()
        self.args = args
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=args.in_channels, out_channels=args.out_channels, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1)
        )

        self.lstm = nn.LSTM(input_size=args.out_channels, hidden_size=args.hidden_size,
                            num_layers=args.num_layers, batch_first=True)
        self.fc = nn.Linear(args.hidden_size, args.output_size)

def forward(self, x):
    x = x.permute(0, 2, 1)
    x = self.conv(x)
    x = x.permute(0, 2, 1)
    x, _ = self.lstm(x)
    x = self.fc(x)
    x = x[:, -1, :]

    return x


In [18]:
Datasets = pd.read_csv('data/Google_Stock_Price_Train.csv')
data = Datasets['Open'].values.astype(float)
train_set = data[:-int(len(data) * 0.2)]
test_set = data[-int(len(data) * 0.2):]
# print(len(train_set))   # 202
# print(len(test_set))    # 50

scaler = MinMaxScaler(feature_range=(-1, 1))
train_norm = scaler.fit_transform(train_set.reshape(-1, 1))
train_norm = torch.FloatTensor(train_norm).view(-1)
train_data = []
seq_size = 9
for i in range(len(train_norm) - seq_size):
    window = train_norm[i:i + seq_size]
    label = train_norm[i + seq_size]
    train_data.append((window, label))

mod = CNN_LSTM(para_dict)

loss_fn = nn.MSELoss()

learning_rate = 1e-2
optimizer = torch.optim.SGD(mod.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(mod.parameters(), lr=learning_rate)
total_train_step = 0
epoch = 500
loss_list = []
mod.train()
for i in range(epoch):
    for seq, y_true in train_data:
        optimizer.zero_grad()
        out = mod(seq.reshape(1, 1, -1))
        loss = loss_fn(out, y_true)
        loss.backward()
        optimizer.step()
        total_train_step = total_train_step + 1
        if total_train_step % 1000 == 0:
            print('训练次数:{}，loss:{}'.format(total_train_step, loss.item()))
            loss_list.append(loss.item())

preds = train_norm[-seq_size:].tolist()
mod.eval()
for i in range(52):
    seq = torch.FloatTensor(preds[-seq_size:])
    with torch.no_grad():
        preds.append(mod(seq.reshape(1, 1, -1)).item())
print(preds)
true_value = scaler.inverse_transform(np.array(preds[seq_size:]).reshape(-1, 1))
print(true_value.tolist())
print(test_set)
plt.figure(1)
plt.plot(test_set, label='true value')
plt.plot(true_value, label='predict value')
plt.legend(loc="upper left")
plt.figure(2)
plt.plot(loss_list, label='loss')
plt.show()


AttributeError: 'dict' object has no attribute 'in_channels'